<a href="https://colab.research.google.com/github/seawavve/test/blob/main/kobert_origin_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경설정 

In [46]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [47]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install transformers==3

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-gvajl624
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-gvajl624
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12773 sha256=3e71c88809981f2c03ee9b08775f2549a573ff9b1936b4a7651bafc60339e09f
  Stored in directory: /tmp/pip-ephem-wheel-cache-mv023zr6/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [48]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [49]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [50]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# 모델

In [51]:
##GPU 사용 시
device = torch.device("cuda:0")

In [52]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [53]:
# !wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
# !wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

In [54]:
dataset_train = nlp.data.TSVDataset("augumented_train_data.tsv", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("test_data.tsv", field_indices=[1,2], num_discard_samples=1)

In [55]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [56]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [57]:
## Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [58]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [59]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [60]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [61]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [62]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [63]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [64]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [65]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [66]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [67]:
import pandas as pd
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

model.eval()
test_eval=[]
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)

    
    for i in out:
        logits=i
        logits = logits.detach().cpu().numpy()
        test_eval.append(np.argmax(logits))

print("test acc {}".format(test_acc / (batch_id+1)))

print(test_eval)
result=pd.DataFrame(test_eval[1:])
result['idx']=range(45654,45654+len(result))
result.set_index('idx',inplace=True)
result.to_csv('result.csv')
display(result)
print(len(result))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.993249773979187 train acc 0.0625
epoch 1 batch id 201 loss 0.539370059967041 train acc 0.5147699004975125
epoch 1 batch id 401 loss 0.43082675337791443 train acc 0.6408977556109726
epoch 1 batch id 601 loss 0.9087293148040771 train acc 0.7007591514143094
epoch 1 batch id 801 loss 0.40997394919395447 train acc 0.7271379525593009
epoch 1 batch id 1001 loss 0.16245143115520477 train acc 0.7453171828171828
epoch 1 batch id 1201 loss 0.6660753488540649 train acc 0.755802456286428
epoch 1 batch id 1401 loss 0.8314477205276489 train acc 0.7651677373304783
epoch 1 batch id 1601 loss 0.7949634790420532 train acc 0.7725445034353529
epoch 1 batch id 1801 loss 0.9483669400215149 train acc 0.7779705719044975
epoch 1 batch id 2001 loss 0.6337947845458984 train acc 0.7823900549725138
epoch 1 batch id 2201 loss 1.2445378303527832 train acc 0.7874545661063153
epoch 1 batch id 2401 loss 0.327791303396225 train acc 0.7917924822990421
epoch 1 batch id 2601 loss 0.2410468459129333

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



test acc 0.11745073108709474
[0, 3, 2, 2, 3, 0, 5, 3, 4, 4, 4, 6, 4, 5, 6, 1, 6, 2, 4, 4, 4, 4, 0, 0, 0, 3, 6, 2, 5, 2, 3, 4, 4, 5, 4, 4, 5, 4, 6, 5, 5, 5, 5, 3, 0, 5, 3, 6, 1, 6, 0, 4, 6, 4, 5, 1, 4, 6, 3, 3, 4, 4, 4, 5, 3, 1, 5, 5, 2, 0, 3, 4, 3, 3, 1, 5, 1, 1, 1, 2, 6, 1, 5, 4, 6, 5, 4, 1, 6, 1, 5, 5, 4, 4, 4, 1, 6, 4, 1, 1, 5, 5, 4, 4, 5, 4, 3, 4, 0, 3, 2, 2, 2, 4, 3, 5, 3, 6, 2, 4, 5, 0, 6, 0, 2, 1, 5, 4, 0, 4, 0, 2, 6, 6, 4, 2, 6, 2, 1, 0, 5, 2, 3, 3, 4, 4, 1, 2, 3, 2, 6, 0, 1, 6, 4, 3, 5, 2, 2, 6, 1, 4, 3, 5, 5, 3, 4, 1, 1, 4, 1, 2, 2, 1, 3, 4, 2, 3, 6, 2, 5, 1, 5, 1, 5, 1, 2, 0, 3, 5, 2, 1, 0, 4, 2, 2, 6, 1, 3, 0, 1, 5, 4, 6, 5, 3, 5, 5, 5, 1, 1, 1, 3, 0, 1, 0, 4, 0, 6, 2, 4, 3, 5, 1, 0, 5, 0, 3, 4, 4, 6, 6, 3, 6, 4, 4, 5, 2, 2, 1, 5, 5, 1, 4, 0, 4, 5, 4, 6, 4, 4, 5, 1, 5, 5, 4, 5, 4, 2, 4, 5, 4, 0, 3, 3, 2, 0, 4, 5, 5, 4, 1, 0, 3, 6, 0, 3, 6, 5, 4, 6, 4, 4, 4, 4, 6, 3, 2, 3, 2, 2, 2, 5, 4, 5, 6, 4, 2, 2, 1, 6, 6, 0, 4, 6, 6, 3, 2, 2, 6, 5, 2, 5, 1, 0, 0, 3, 5, 2, 6, 5, 4, 1, 

,0
idx,
45654,3
45655,2
45656,2
45657,3
45658,0
...,...
54779,3
54780,2
54781,2


9130
